<a href="https://colab.research.google.com/github/kavlata/LLM-experiments/blob/main/HF_MCQA_finetuning_coveo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets transformers
!pip install accelerate -U
!pip install transformers[torch]
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import pandas as pd
import csv
path1 = "/content/sample_data/train.csv"
df_train = pd.read_csv(path1, quoting=csv.QUOTE_MINIMAL)
df_train.head()

,idx,text,choice1,choice2,choice3,choice4,choice5,choice6,label
0,0,"The Rhodesian Bush War was the result , begin...",fought,contended,defended,opposed,NaN,NaN,choice1
1,1,"<ArrayOfQuery xmlns: xsi=""http://www.w3.org/20...",instance,case,illustration,example,NaN,NaN,choice1
2,2,<! -- Beginning of the Coveo Searchbox [BLANK]...,segment,incision,section,subdivision,NaN,NaN,choice3
3,3,"<div class=""CoveoFacet"" data-title=""Salesforce...",Level,Leg,Degree,Stage,NaN,NaN,choice4
4,4,The Sitecore Security Provider ABC has encount...,year,course,class,family,NaN,NaN,choice3


In [3]:
df_train['choice1'].fillna("", inplace = True)
df_train['choice2'].fillna("", inplace = True)
df_train['choice3'].fillna("", inplace = True)
df_train['choice4'].fillna("", inplace = True)
df_train['choice5'].fillna("", inplace = True)
df_train['choice6'].fillna("", inplace = True)

In [4]:
def assign_labelid(score):
    if score == 'choice1':
        return 1
    elif score == 'choice2':
        return 2
    elif score == 'choice3':
        return 3
    elif score == 'choice4':
        return 4
    elif score == 'choice5':
        return 5
    else:
        return 6

# Apply the function to create a new 'Grade' column
df_train['label_id'] = df_train['label'].apply(assign_labelid)
print(df_train.iloc[0][['label','label_id']])

label       choice1
label_id          1
Name: 0, dtype: object


In [5]:
model_checkpoint = "bert-base-uncased"
batch_size = 4
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [6]:
import numpy as np
rows_per_group = 150

# Divide the DataFrame into groups
groups = np.array_split(df_train[0:300], 300 // rows_per_group)

# Reset the index of each group to begin from 0
groups = [group.reset_index(drop=True) for group in groups]

df_train_150 = groups[0].drop('idx',axis=1)
df_test_150 = groups[1].drop('idx',axis=1)


# # Print the groups
# for i, group in enumerate(groups):
#     print(f"Group {i+1}:")
#     # print(group)
#     print()


In [7]:
df_train_5 = df_train[0 : 5]
split_names1 = df_train_5['text'].str.split('\[BLANK\]', n=1, expand=True)
split_names1.columns = ['sent1', 'sent2']
# Merge the split names with the original DataFrame
df_train_5 = pd.merge(df_train_5, split_names1, left_index=True, right_index=True)


split_names2 = df_train_150['text'].str.split('\[BLANK\]', n=1, expand=True)
split_names2.columns = ['sent1', 'sent2']
# Merge the split names with the original DataFrame
df_train_150 = pd.merge(df_train_150, split_names2, left_index=True, right_index=True)


split_names3 = df_test_150['text'].str.split('\[BLANK\]', n=1, expand=True)
split_names3.columns = ['sent1', 'sent2']
# Merge the split names with the original DataFrame
df_test_150 = pd.merge(df_test_150, split_names3, left_index=True, right_index=True)


In [8]:
ending_names = ["choice1","choice2","choice3","choice4","choice5","choice6"]
def preprocess_function(examples, choiceCount=6):

    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * choiceCount for context in examples["sent1"]]
    # Grab all second sentences possible for each context.
    question_headers = examples["sent2"]
    second_sentences = [[f"{examples[end][i]} [SEP]{header}" for end in ending_names] for i, header in enumerate(question_headers)]

    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+choiceCount] for i in range(0, len(v), choiceCount)] for k, v in tokenized_examples.items()}

In [9]:
model_checkpoint = "bert-base-uncased"
batch_size = 4
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 

In [10]:
import torch
# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"/content/sample_data/{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False
)

In [11]:
df_train_150 = df_train_150.rename(columns={'label': 'groundtruth'})
df_train_150 = df_train_150.rename(columns={'label_id': 'label'})
df_test_150 = df_test_150.rename(columns={'label': 'groundtruth'})
df_test_150 = df_test_150.rename(columns={'label_id': 'label'})
print(df_train_150.loc[0])
print(df_test_150.loc[0])

text            The Rhodesian Bush War was the result , begin...
choice1                                                   fought
choice2                                                contended
choice3                                                 defended
choice4                                                  opposed
choice5                                                         
choice6                                                         
groundtruth                                              choice1
label                                                          1
sent1           The Rhodesian Bush War was the result , begin...
sent2           back strongly , reducing the number of guerri...
Name: 0, dtype: object
text           QRE is not working as expected | Coveo Answers...
choice1                                                   Motion
choice2                                                 Question
choice3                                            Interrogation
ch

In [12]:
import datasets
from datasets import  Dataset, DatasetDict
dataset_train = Dataset.from_pandas(df_train_150)
dataset_test = Dataset.from_pandas(df_test_150)
dataset_dict = DatasetDict({'train': dataset_train, 'test': dataset_test})
# Access the converted dataset
encoded_datasets = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [13]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        print(label_name)
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [14]:
# features_train = encoded_datasets["train"]
accepted_keys = ["input_ids", "attention_mask", "label"]

features_c = [{k: v for k, v in encoded_datasets['train'][i].items() if k in accepted_keys} for i in range(10)]
# print(features_c[0])
batch = DataCollatorForMultipleChoice(tokenizer)(features_c)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


label


In [15]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [16]:

import torch
import time
print(torch.version)
print(torch.cuda.is_available())
device = torch.device('cuda:1')
args.device

<module 'torch.version' from '/usr/local/lib/python3.10/dist-packages/torch/version.py'>
True


device(type='cuda', index=0)

In [17]:
# encoded_datasets["train"] = features_train
#encoded_datasets["validation"] = features_test
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,

)

In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


label


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.785021,0.286667
2,No log,1.786071,0.286667
3,No log,1.729704,0.273333


label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
label
labe

TrainOutput(global_step=114, training_loss=1.7616068522135417, metrics={'train_runtime': 85.512, 'train_samples_per_second': 5.262, 'train_steps_per_second': 1.333, 'total_flos': 192718895469984.0, 'train_loss': 1.7616068522135417, 'epoch': 3.0})

In [19]:
output_dir = "/content/sample_data/"
trainer.save_model(output_dir)


In [20]:
from transformers import AutoModelForMultipleChoice

model = AutoModelForMultipleChoice.from_pretrained("/content/sample_data/")

In [32]:
prompt = "The Rhodesian Bush War was the result , beginning in earnest on 21 December 1972 when ZANLA attacked Altena and Whistlefield Farms near Centenary in the country 's north @-@ east . The Rhodesian Security Forces launched Operation Hurricane in response and [BLANK] back strongly , reducing the number of guerrillas active within the country to under 300 by December 1974 . In the period October â€“ November 1974 , they killed more nationalist fighters than in the previous two years combined . However , a South African @-@ brokered ceasefire , which the security forces respected and the insurgents ignored , helped the nationalists to win back ground . By 1977 there were 2 @,@ 500 guerrillas operating in Rhodesia , with several times that number in training abroad ."
choice0 = "fought"
choice1 = "contended"
choice2 = "defended"
choice3 = "opposed"

labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1

encoding = tokenizer([prompt, prompt,prompt,prompt], [choice0, choice1,choice2,choice3], return_tensors="pt", padding=True)
outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=labels)  # batch size is 1

# the linear classifier still needs to be trained
loss = outputs.loss
logits = outputs.logits

In [22]:
answer_choices = [choice0,choice1, choice2,choice3]
predicted_answer_index = torch.argmax(logits, dim=1).item()
predicted_answer = answer_choices[predicted_answer_index]

print("Predicted Answer:", predicted_answer)

Predicted Answer: contended


In [40]:
torch.argmax(logits, dim=1).item()

1

In [ ]:
encoded_text = tokenizer(prompt, return_tensors="pt")

#1. step to get the logits of the next token
with torch.inference_mode():
  outputs = model(**encoded_text)

In [33]:
next_token_logits = outputs.logits[0, :]
print(next_token_logits.shape)
print(next_token_logits)



torch.Size([4])
tensor([-0.7135, -0.5507, -0.7548, -0.5834], grad_fn=<SliceBackward0>)


In [38]:
# 2. step to convert the logits to probabilities
next_token_probs = torch.softmax(next_token_logits, -1)

# 3. step to get the top 10
topk_next_tokens= torch.topk(next_token_probs, 4)



In [39]:
#putting it together
print(*[(tokenizer.decode(idx), prob) for idx, prob in zip(topk_next_tokens.indices, topk_next_tokens.values)], sep="\n")

('[unused0]', tensor(0.2753, grad_fn=<UnbindBackward0>))
('[unused2]', tensor(0.2664, grad_fn=<UnbindBackward0>))
('[PAD]', tensor(0.2339, grad_fn=<UnbindBackward0>))
('[unused1]', tensor(0.2244, grad_fn=<UnbindBackward0>))
